# 项目主要内容

## 开源可视化应用项目链接：[https://aistudio.baidu.com/application/detail/94699](https://aistudio.baidu.com/application/detail/94699)

<div align="center"><iframe src="//player.bilibili.com/player.html?isOutside=true&aid=115180453299696&bvid=BV1gHHRzVEPi&cid=32296079129&p=1" style="width:40vw; aspect-ratio:16/9; max-width:100%;" scrolling="no" border="0" frameborder="no" framespacing="0" allowfullscreen="true"></iframe></div>

- **开源源码**：网页端源码完全开源~训练过程也开源哦，欢迎大家根据我们的页面进行修改创建新的属于自己的网页
```
git lfs install
git clone https://git.aistudio.baidu.com/testlyh/teavisor.git
```
- **yoloe模型构建**：使用roboflow的茶叶病虫害(tea disease)进行coco数据集构建与扩容，构建了[茶叶病虫害数据集](https://aistudio.baidu.com/dataset/detail/355119/intro)，并使用**PaddleDetection**进行[PP-YOLOE的训练](https://aistudio.baidu.com/project/edit/9475446)

- **天气信息获取**：借助百度云千帆的天气收集组件（**组件id：c-wf-933c4600-dad6-4349-8b62-2c2c4abedbdb**），获取指定地区天气。通过结合天气信息，能够为茶农或茶叶医生提供更加现实，更加顺应天时的茶园治理方案

- **多模态大模型微调**：利用**ERNIEkit**进行多模态模型的微调，利用kaggle开源病虫害目标识别数据集archive/tea sickness dataset，并使用飞桨社区的ERNIE-4.5-VL-424B-A47B-Paddle大模型，生成[茶叶病虫害微调数据集](https://aistudio.baidu.com/dataset/detail/355134/intro)，再对小模型ERNIE-4.5-VL-28B-A3B-Paddle进行LoRA微调（由于飞桨社区A800只有单卡，ERNIEkit不支持多模态大模型使用单卡微调；而V100多卡架构过老，ERNIEkit不支持该型号显卡多卡微调，因此此模型微调使用本地资源进行，微调文件为finetune.ipynb,需要安装flash-attn），**使得小模型能够更加精准的识别病虫害并描述当前叶片病虫害状况**，帮助后续接入百度搜索的文心文本大模型（ernie-4.5-turbo-128k-preview）能够更好输出防治建议

- **语音转写与语音合成**：调用百度云的api（ai.baidu.com），将用户语音转写为文字后再输入到模型当中，即使是文化水平不高的茶农，可以直接与ai进行对话。并利用百度云的语音合成api，使用**度阿闽-闽南男声**进行输出，可以直接听到所有防治建议。语音转写与合成打破了传统大模型需要认字识字进行输入的壁垒，**帮助茶农户更加方便与低成本的获取到专业建议**

**用户上传图片后，整体技术路线图**：

<div align="center">
  <img src="https://ai-studio-static-online.cdn.bcebos.com/2e87f1731ff248e297820d05ce8c9cc918cb93e357b24f348cbf2bc35aae09db" alt="图片描述" style="max-width: 100%;">
</div>

# 项目背景 
**茶叶不仅是中国农业文明和文化的重要载体，也是具有全球竞争力的战略性产业。**
然而，目前我国茶园在种植环节的数字化、机械化和智能化水平整体偏低，全国茶园综合机械化率不足30%。病虫害识别主要依赖人工经验，效率低、误差大，农药使用缺乏科学指导，既影响茶叶品质和出口安全，也增加了生产成本。在这样的背景下，茶园管理仍以“小农户分散经营”为主，且茶农多为中老年人，文化水平普遍偏低，资金和技术条件有限，难以承担高成本的智能系统，因此迫切需要一种低门槛、易推广、可落地的病虫害防治工具。

# 项目目标
围绕茶叶种植过程中病虫害识别和农事决策的实际需求，团队拟开发一套集图像识别和信息服务于一体的轻量化系统，融合“**多模态模型、目标检测与网页平台**”技术，让茶农与茶叶医生只需“用手机拍照→快速诊断→获取治理建议”，即可完成病虫害识别和防治决策。该系统将有效降低生产和用药成本，减少农药滥用，提升茶叶品质，推动茶产业绿色、高质量发展。



# 大模型微调详解

基于**ERNIEkit**给出的模板，构建了**茶叶多模态大模型问答数据集**，可以帮助小模型向大模型学习（ERNIE4.5-VL-28B 向他的非量化大模型进行学习）
如果有多卡A系列显卡（如A100），可以使用如下代码进行微调
> 请一定要注意：ERNIEkit的**多模态模型微调**，不支持**单卡**微调，只支持多卡微调，这里给出两种微调方法
> 请再次注意！**ERNIEkit中的部分库还只支持Linux下载**，所以windows使用ERNIEkit进行微调可能会在安装requirement的时候就直接卡住（可以尝试删除卡住的库的内容
>  还要注意，gradient_checkpointing=False，ERNIE4.5-vl不支持梯度检查，这意味着省不了显存了QAQ

## 多卡Linux服务器（且显卡不能太老，ERNIEkit也不支持服务器的V100），使用ERINEkit
### 下载ERNIEkit
```
git clone https://gitee.com/paddlepaddle/ERNIE.git
cd /home/aistudio/ERNIE
python -m pip install -r requirements/gpu/requirements.txt
python -m pip install -e . # 推荐使用可编辑模式安装
```
### 将ERINEkit添加到环境变量，然后启动训练脚本
```
export PATH=$PATH:/home/aistudio/external-libraries/bin  && erniekit train /home/aistudio/run_sft_8k.yaml
```
**这里一定要将你的bin文件夹加入到环境变量！！！**（这都是我踩的坑）
> 如果提示你erniekit不是可执行的命令，就说明你没有将指定的文件夹加入到环境变量当中，如果你想在本地跑，需要这么做
- linux： 找到环境下的bin文件夹，将其加入到环境变量
- windows：找到环境下的Script文件夹，将其加入到环境变量

### 训练
[点击这里跳转到github下载配置yaml文件](https://github.com/PaddlePaddle/ERNIE/tree/develop/examples/configs/ERNIE-4.5-VL-28B-A3B/sft)
这里训练需要构建一个yml,在markdown中微调项目链接是我已经构建好的yml，，如果要在本地运行，需要手动打开将其中的以下路径进行替换
> model_name_or_path: /home/aistudio/data/models/30648/ERNIE-4.5-VL-28B-A3B-Paddle
train_dataset_path: /home/aistudio/data/datasets/355134/vl_finetune/sft_vl_train_shuffle.jsonl

同时训练也支持量化，可以添加quantity参数进行量化

## 单卡情况下使用transformer加载模型并进行训练（下载本项目下的finetune.ipynb）
需要下载本项目下的notebook，之后修改一些参数
如果资源充足还可以额外增加LoRA插入的层，效果会更好
### 注意事项
**如果你想要在脚本里指定使用某张显卡，必须在导入torch之前，就把环境变量指定好！！！**，这是最关键的，因为先导入torch再设置环境变量只能限制显卡的个数，而不能限制使用显卡的gpuid了
```python
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 这样能够指定只有一块gpu并使用gpuid为1的gpu
import torch
```
```python
# 错误示范
import os
import torch  # 此时会将所有device都加入
os.environ["CUDA_VISIBLE_DEVICES"] = "4"  # 这样会导致只有一张gpu能够被当前脚本所使用，但由于torch已经加载了所有gpu，所以这里就会直接使用gpuid=0的卡！！！导致指定gpu运行失败 
```

### 需要修改的常数
```python 
model_path = 改成你下载的模型的文件夹路径
data_file = 改成数据集中的jsonl文件路径
image_dir = 改成数据集中的存放所有图片的文件夹路径
output_dir = "改成你希望的结果输出文件夹
num_epochs = 10
batch_size = 2
gradient_accumulation_steps = 16  # 梯度累计清空所积累的步数，这里梯度将会累计16次后再反向传播
learning_rate = 3e-4

# 下面的加速器代码，服务器A100如果用了会直接爆显存，痛苦的oom
# accelerator = Accelerator()
# device = accelerator.device
```